In [ ]:
try:
  # This command only in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Get project files
#!wget https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip

#!unzip cats_and_dogs.zip
PATH = r'C:\Users\udy\.keras\datasets\cats_and_dogs'
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

# Get number of files in each directory. The train and validation directories
# each have the subdirecories "dogs" and "cats".
total_train = sum([len(files) for r, d, files in os.walk(train_dir)])
total_val = sum([len(files) for r, d, files in os.walk(validation_dir)])
total_test = len(os.listdir(test_dir))

# Variables for pre-processing and training.
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [ ]:
# 3
test_subdir = os.path.join(test_dir, 'unknown')
if not os.path.exists(test_subdir):
    os.makedirs(test_subdir)
for fyl in os.listdir(test_dir):
    if fyl[-3:] == 'jpg':
        os.rename(os.path.join(test_dir, fyl), os.path.join(test_subdir, fyl))

train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0 / 255.0)
validation_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0 / 255.0)
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0 / 255.0)

train_data_gen = train_image_generator.flow_from_directory(directory = train_dir, batch_size = batch_size, target_size = (IMG_HEIGHT, IMG_WIDTH), class_mode = 'binary')
val_data_gen = validation_image_generator.flow_from_directory(directory = validation_dir, batch_size = batch_size, target_size = (IMG_HEIGHT, IMG_WIDTH), class_mode = 'binary')
test_data_gen = test_image_generator.flow_from_directory(directory = test_dir, batch_size = batch_size, target_size = (IMG_HEIGHT, IMG_WIDTH), shuffle = False, class_mode = None)

In [ ]:
# 4
def plotImages(images_arr, probabilities = False):
    fig, axes = plt.subplots(len(images_arr), 1, figsize=(5,len(images_arr) * 3))
    if probabilities is False:
      for img, ax in zip( images_arr, axes):
          ax.imshow(img)
          ax.axis('off')
    else:
      for img, probability, ax in zip( images_arr, probabilities, axes):
          ax.imshow(img)
          ax.axis('off')
          if probability > 0.5:
              ax.set_title("%.2f" % (probability*100) + "% dog")
          else:
              ax.set_title("%.2f" % ((1-probability)*100) + "% cat")
    plt.show()

sample_training_images, _ = next(train_data_gen)
plotImages(sample_training_images[:5])


In [ ]:
# 5
train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range = 30,
                                                                        width_shift_range = 0.1,
                                                                        height_shift_range = 0.1,
                                                                        zoom_range = 0.2,
                                                                        horizontal_flip = True,
                                                                        fill_mode = 'nearest',
                                                                        rescale = 1.0 / 255.0)

In [ ]:
# 6
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

plotImages(augmented_images)

In [ ]:
# 7
# this model consistently gets > 70% accuracy. it can be further improved by experimenting on model's design
model = Sequential()
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5, 5), activation = 'relu', input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)))    # input layer with 32 filters of size (5, 5). data input shape (IMG_HEIGHT, IMG_WIDTH, 3)
model.add(tf.keras.layers.MaxPooling2D(pool_size = (4, 4)))    # pooling with Max function with window size (4, 4). strides default to pool_size if not mentioned or None
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (4, 4), activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (3, 3)))
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (2, 2), activation = 'relu'))
model.add(tf.keras.layers.Flatten())    # to flatten the multidimensional data
model.add(tf.keras.layers.Dense(64, activation = 'relu', kernel_initializer = 'he_uniform'))    # dense hidden layer; he_uniform is recommended with relu
model.add(tf.keras.layers.Dense(1))    # output layer

model.compile(optimizer = tf.keras.optimizers.Adam(),    # learning_rate can be passed to Adam if needed
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

model.summary()

In [ ]:
# 8
history = model.fit(train_data_gen,
                    epochs = epochs,
                    validation_data = val_data_gen)

In [ ]:
probabilities = np.round(tf.nn.sigmoid(model.predict(test_data_gen)).numpy().flatten()).astype(np.int64)

In [ ]:
# 9
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# 11
answers =  [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
            1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
            1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
            1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
            0, 0, 0, 0, 0, 0]

correct = 0

for probability, answer in zip(probabilities, answers):
  if round(probability) == answer:
    correct +=1

percentage_identified = (correct / len(answers)) * 100

passed_challenge = percentage_identified >= 63

print(f"Your model correctly identified {round(percentage_identified, 2)}% of the images of cats and dogs.")

if passed_challenge:
  print("You passed the challenge!")
else:
  print("You haven't passed yet. Your model should identify at least 63% of the images. Keep trying. You will get it!")